# 02_hyphenation

> Hyphenator

In [ ]:
#| default_exp hyphenation

In [ ]:
#| hide
from nbdev.showdoc import *
import re
import itertools as it

In [ ]:
#| exporti
def add_hyphens(s, positions, hyphen='-'):
    i0, i1 = it.tee(iter(positions))
    i0 = it.chain((0,), i0)
    i1 = it.chain(i1, (len(s),))
    substrings = (s[p0:p1] for (p0,p1) in zip(i0, i1))
    return hyphen.join(substrings).strip(hyphen)

In [ ]:
assert add_hyphens('saippuakauppias', ()) == 'saippuakauppias'
assert add_hyphens('saippuakauppias', (7,)) == 'saippua-kauppias'
assert add_hyphens('saippuakauppias', (4, 7, 11)) == f'saip-pua-kaup-pias'
assert add_hyphens('', ()) == ''

In [ ]:
#| export
def hyphenator(regex, mapping, exceptions, hyphen='-'):
    def fun(word):
        if (result := exceptions.get(word)):
            return result
        word = f'\x1f{word}\x1f'
        weights = bytearray(len(word))
        for match in regex.finditer(word):
            pos = match.span()[0]-1
            key = match.group(1)
            rule = mapping[key]
            for i, w in enumerate(rule):
                weights[pos+i] = max(weights[pos+i], w)
        return add_hyphens(word[1:-1], (i for (i,w) in enumerate(weights) if w&1==1), hyphen=hyphen)
    return fun

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()